In [1]:
from huggingface_hub import login
from datasets import load_dataset
import os
from icecream import ic

login(os.environ['HF_TOKEN'])

data_name = 'pgajo/EW-TT-PE_en-it_spaced'
dataset = load_dataset(data_name)

ic(dataset)

raw_labels = []
from tqdm.auto import tqdm
for line in tqdm(dataset['train']):
    # print(line)
    for result in line['predictions'][0]['result']:
        if result['value']['labels'][0] not in raw_labels:
            raw_labels.append(result['value']['labels'][0])
print(raw_labels)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
example = dataset['train'][0]

ic(example)

example_text = example['data']['text_en']
example_text
tokens = tokenizer(example_text, return_tensors='pt')

ic(tokens)

label_list = []
input_ids = tokens['input_ids'].squeeze()
for i, id in enumerate(input_ids):
    span = tokens.token_to_chars(i)
    if int(id) not in [101, 102]:
        start = span.start
        end = span.end
        ic(start, end)
        ic(example['data']['text_en'][start:end])
        for result in example['predictions'][0]['result']:
            if start >= result['value']['start'] and end <= result['value']['end']:
                new_item = 'B-'+result['value']['labels'][0]
                label_list.append(new_item)
        # ic(i, int(id), new_item, example['data']['text_en'][start:end])
    else:
        new_item = -100
        label_list.append(new_item)
ic(len(label_list))
ic(len(input_ids))
# for a, b in zip(tokens['input_ids'].squeeze(), label_list):
#     ic(a,b)


/opt/miniconda3/envs/food/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /models/huggingface/token
Login successful


DatasetDict({
    train: Dataset({
        features: ['data', 'predictions'],
        num_rows: 630
    })
    test: Dataset({
        features: ['data', 'predictions'],
        num_rows: 70
    })
})